In [1]:
import pandas as pd
import numpy as np
from dplython import (DplyFrame, X, select, sift, sample_n,
    sample_frac, head, arrange, mutate, group_by, summarize, DelayFunction)

In [2]:
# dplython different/more intuitive syntax for manipulating data
combine = '/Users/katbishop/Desktop/DSI-SF2-bishopkd/datasets/football_combine/combine.csv'

In [5]:
combine = DplyFrame(pd.read_csv(combine))
combine.head()

/Users/katbishop/anaconda/envs/dsi/lib/python2.7/site-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/Users/katbishop/anaconda/envs/dsi/lib/python2.7/site-packages/IPython/core/formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/Users/katbishop/anaconda/envs/dsi/lib/python2.7/site-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/Users/katbishop/anaconda/envs/dsi/lib/python2.7/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/Users/kat

,id,Year,Name,Position,HeightFeet,HeightInches,Weight,Arms,Hands,FortyYD,...,Bench,Round,College,Pick,PickRound,PickTotal,FirstName,LastName,HeightInchesTotal,Wonderlic
0,8984,2013,Quanterus Smith,DE,6,5.0,250,33.25,10.375,0.0,...,0,5,Western Kentucky,13(146),13,146,Quanterus,Smith,77.0,0
1,9002,2013,Abry Jones,DT,6,3.0,313,35.00,9.750,0.0,...,30,0,NaN,NaN,0,0,Abry,Jones,75.0,0
2,9004,2013,Bennie Logan,DT,6,2.0,309,34.00,10.250,0.0,...,30,3,LSU,5(67),5,67,Bennie,Logan,74.0,0
3,9012,2013,John Boyett,FS,5,10.0,204,30.50,8.500,0.0,...,27,6,Oregon,24(192),24,192,John,Boyett,70.0,0
4,9018,2013,Bacarri Rambo,FS,6,0.0,211,31.00,9.250,0.0,...,17,6,Georgia,23(191),23,191,Bacarri,Rambo,72.0,0


In [8]:
# position = combine['position'] -- undply syntax
position = combine >> select(X.Position)
print position[0:5]


  Position
0       DE
1       DT
2       DT
3       FS
4       FS


In [9]:
# names of players who weight is < 200
# pandas: 
players = combine[['Name','Weight']]
players = players[players.Weight < 200]
players.head()

,Name,Weight
15,Brandon Boykin,182
49,Foswhitt Whittaker,193
55,Ryan Broyles,192
56,Chase Minnifield,183
58,Shaun Prater,190


In [10]:
# dplython
dply_players = (combine >> sift(X.Weight < 200) >> select(X.Name, X.Weight))
dply_players.head()

,Name,Weight
15,Brandon Boykin,182
49,Foswhitt Whittaker,193
55,Ryan Broyles,192
56,Chase Minnifield,183
58,Shaun Prater,190


In [12]:
# way to take a random sample 0f 5% and take out the Name and Weight
# pandas
sample = combine[['Name','Weight']]

sample = combine.sample(frac=.05)
sample.head(2)

,id,Year,Name,Position,HeightFeet,HeightInches,Weight,Arms,Hands,FortyYD,...,Bench,Round,College,Pick,PickRound,PickTotal,FirstName,LastName,HeightInchesTotal,Wonderlic
2394,12526,2001,Marques Tuiasosopo,QB,6,1.0,223,0.0,0.0,4.71,...,0,2,Washington,28(59),28,59,Marques,Tuiasosopo,73.0,24
689,12887,1999,Donald Mitchell,CB,5,10.0,187,0.0,0.0,4.47,...,24,4,Southern Methodist,22(117),22,117,Donald,Mitchell,70.0,0


In [13]:
# dply; don't have to declare df cols first
# leave out select portion to select all cols

dply_sample = (combine >> sample_frac(.05) >> select (X.Name, X.Weight))
dply_sample.head(2)

,Name,Weight
4559,Adam Grant,325
3788,Kelly Butler,324


In [17]:
# group by Position and Year, add together weight and arms,
# and return the mean of that column

# pandas
def weight_plus_arms_mean(df):
    summed = df['Weight'] + df['Arms']
    df['weight_arms_mean'] = np.mean(summed)
    return df.loc[:,'weight_arms_mean'].iloc[0]

combine.groupby(['Position', 'Year']).apply(weight_plus_arms_mean).reset_index().head(3)




,Position,Year,0
0,C,2014,307.166667
1,CB,1999,184.807692
2,CB,2000,191.750000


In [24]:
# dply
# mutate - add cols
weight_arms_mean = (combine >> group_by(X.Position, X.Year) >> 
                    mutate(weight_arms_mean=X.Weight + X.Arms) >> 
                    summarize(weight_arms_mean=X.weight_arms_mean.mean(),
                             year_mean=X.Year.mean()))
weight_arms_mean.head(3)

,Position,Year,weight_arms_mean,year_mean
0,C,2014,307.166667,2014
1,CB,1999,184.807692,1999
2,CB,2000,191.750000,2000


In [22]:
@DelayFunction
def mean_of_columns(df):
    meancol = df.mean(axis=1)
    return meancol

# mean of year, weight, and hands columns:
ywh_mean = (combine >>
            select (X.Name, X.Weight, X.Year, X.Hands) >>
            mutate(mean_of_ywh = (combine >>
                                 select(X.Weight, X.Year, X.Hands) >>
                                 mean_of_columns(X))))
ywh_mean.head(3)

,Name,Weight,Year,Hands,mean_of_ywh
0,Quanterus Smith,250,2013,10.375,757.791667
1,Abry Jones,313,2013,9.750,778.583333
2,Bennie Logan,309,2013,10.250,777.416667
